In [1]:
%load_ext dotenv
%dotenv

import os

%cd {os.getenv("PROJECT_PATH") or "."}

%load_ext autoreload
%autoreload 1

from IPython.display import display

/home/aris/projects/soc_molgen


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
from pathlib import Path
from absl import logging
from tqdm.notebook import tqdm, trange
from timeit import default_timer as timer
import pickle
from collections import defaultdict

logging.set_verbosity(logging.INFO)

In [3]:
from pandarallel import pandarallel

pandarallel.initialize(
    nb_workers=os.cpu_count(),
    progress_bar=True,
    verbose=0
)

In [4]:
def show_df(df: pd.DataFrame):
    display(df.head())
    print(df.shape)

In [5]:
from rdkit import Chem

from src.utils.scores import *
from src.vae import load_vae
from src.pinn.pde import load_wavepde
from src.pinn import VAEGenerator

Found local copy...
Found local copy...
Found local copy...


In [6]:
# df = pd.read_csv('data/interim/corr/hj_unsup_0.1_binding_affinity.csv', index_col=0)
df = pd.read_csv('data/interim/corr/wave_unsup_0.1.csv', index_col=0)
# df = pd.read_csv('data/interim/corr/vaepde_unsup_0.1.csv', index_col=0)

# df['1err'] = df['1err'].apply(lambda x: min(x, 0))
# df['2iik'] = df['2iik'].apply(lambda x: min(x, 0))

show_df(df)

,k_idx,idx,t,smiles,plogp,sa,qed,drd2,jnk3,gsk3b,uplogp
0,0,0,0,COCCNCCCN=CNNC1CNC2N3CN=CC=CN1ONC=CNN2C#C3,-39.530140,7.182212,0.073608,0.013799,0.06,0.08,-11.428712
1,0,1,0,COCC(N)NC=CN=CNC=NC=C=CNCN=CSN=C[123I],-4.947651,6.435193,0.061780,0.013203,0.04,0.11,-5.497393
2,0,2,0,COCCNC=NNC=CNSC1NN=CN=C2OC=CN=C1NNNCN=C2C=CCO,-31.870318,6.874465,0.054739,0.018649,0.10,0.12,-10.693365
3,0,3,0,COCCCCCC1N=CN2S3(=C)C=COOC1CONNCC=C3N2,-30.531147,7.382770,0.232168,0.006701,0.04,0.14,-10.406170
4,0,4,0,COCCNCCN(C=NCC=NC1=C=COC=CN)COC(N)C=C=[C@@H1]N=C1,-36.520394,6.024698,0.296935,0.014221,0.05,0.12,-6.946098


(100000, 11)


In [23]:
# PROPS = ["plogp", "sa", "qed", "drd2", "jnk3", "gsk3b", "uplogp"]
PROPS = ['1err', '2iik']

In [24]:
from scipy.stats import pearsonr, spearmanr, linregress

def fun(x:pd.DataFrame):
    results = []
    # for prop in PROPS:
    # for j in range(4, 11):
    for j in range(5, 7):
        r = [
            x.iloc[i, j] for i in range(10)
        ]
        results.append(pearsonr(range(10), r)[0])
    s = pd.Series(results, index=PROPS)
    
    s['invalid'] = x['smiles'].nunique()==1
    
    return s

_df = df.groupby(['k_idx', 'idx']).apply(fun)
# _df = df.groupby(['k_idx', 'idx']).apply(fun)

show_df(_df)

1err      2iik  invalid
k_idx idx                             
0     0    0.347328  0.740238    False
      1   -0.524071 -0.583337    False
      2    0.561304  0.044214    False
      3   -0.452046 -0.494362    False
      4   -0.328536 -0.328957    False

(320, 3)


In [25]:
# df_result = _df.query('not invalid').fillna(0)
df_result = _df.fillna(0)

# show_df(df_result)

print(df_result.shape)

(320, 3)


In [26]:
df_result.groupby('k_idx').mean()

,1err,2iik,invalid
k_idx,,,
0,-0.035357,0.044379,0.00000
1,0.136023,0.117031,0.00000
2,0.061426,0.170072,0.00000
3,0.031710,-0.011199,0.03125
4,0.068465,0.100757,0.00000
5,0.044266,0.006321,0.53125
6,-0.122531,0.093452,0.00000
7,0.129196,0.172424,0.00000
8,-0.011588,0.121669,0.06250
